# HW3 Image Classification
## We strongly recommend that you run with Kaggle for this homework
https://www.kaggle.com/c/ml2022spring-hw3b/code?competitionId=34954&sortBy=dateCreated

# Get Data
Notes: if the links are dead, you can download the data directly from Kaggle and upload it to the workspace, or you can use the Kaggle API to directly download the data into colab.


In [ ]:
#! wget https://www.dropbox.com/s/6l2vcvxl54b0b6w/food11.zip
# ! wget -O food11.zip "https://github.com/virginiakm1988/ML2022-Spring/blob/main/HW03/food11.zip?raw=true"

In [ ]:
# ! unzip food11.zip

# Training

In [1]:
_exp_name = "sample"

In [1]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import torch.hub
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [2]:
myseed = 3231  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)
# "cuda" only when GPUs are available.
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
print(device)

cuda:0


## **Transforms**
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [4]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.RandomAffine(degrees=18, translate=(0.15, 0.15), scale=(0.8, 1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(),
    transforms.ToTensor(),
])


## **Datasets**
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [5]:
def split_set(path, ratio = 0.8, files = None, shuffle = True):
    
    files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
    if shuffle == True:
        random.shuffle(files)
    if ratio < 1:
        train_image_paths, valid_image_paths = files[:int(ratio*len(files))], files[int(ratio*len(files)):]    
        return train_image_paths, valid_image_paths
    elif ratio == 1:
        test_image_paths = files[:]
        return test_image_paths

In [6]:
class FoodDataset(Dataset):

    def __init__(self,pathes,tfm=test_tfm):
        super(FoodDataset).__init__()
        self.pathes = sorted(pathes)
        print(f"dataset with {len(self.pathes)} images")
        self.transform = tfm
  
    def __len__(self):
        return len(self.pathes)
  
    def __getitem__(self,idx):
        fname = self.pathes[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("\\")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label


## **Models**

In [7]:
se_resnet50 = torch.hub.load(
    'moskomule/senet.pytorch',
    'se_resnet50',
    pretrained=False, num_classes = 11).to(device)

Using cache found in C:\Users\User/.cache\torch\hub\moskomule_senet.pytorch_master


# Train

In [9]:
batch_size = 128
_dataset_dir = "./food11"
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
# train_set = FoodDataset(os.path.join(_dataset_dir,"training"), tfm=train_tfm)
# valid_set = FoodDataset(os.path.join(_dataset_dir,"validation"), tfm=test_tfm)
train_image_paths, valid_image_paths = split_set(os.path.join(_dataset_dir,"all"), ratio = 0.85)


train_set = FoodDataset(train_image_paths,tfm=train_tfm)
train_set2 = FoodDataset(train_image_paths,tfm=test_tfm)

train_set_all = ConcatDataset([train_set,train_set2])
print('training set:',len(train_set_all))
valid_set = FoodDataset(valid_image_paths,tfm=test_tfm)

train_loader = DataLoader(train_set_all, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)


dataset with 11301 images
dataset with 11301 images
training set: 22602
dataset with 1995 images


In [10]:
def mixup_data(x, y, alpha=1.0):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [11]:
# The number of training epochs and patience.
n_epochs = 3000
patience = 300 # If no improvement in 'patience' epochs, early stop
learning_rate = 1e-3
# Initialize a model, and put it on the device specified.
# model = Classifier().to(device)

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.0003, weight_decay=1e-5) 
optimizer = torch.optim.AdamW(se_resnet50.parameters(), betas=(0.9, 0.98), lr=learning_rate, weight_decay=0.005)
# optimizer = torch.optim.SGD(se_resnet50.parameters(), lr=learning_rate, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor = 0.5, patience = 20, min_lr = 0.0001)

In [14]:
# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0
# se_resnet50.load_state_dict(torch.load("./se_resnet50_best.ckpt"))
for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    se_resnet50.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)
        inputs, targets_a, targets_b, lam = mixup_data(imgs, labels,alpha = 1)
#         inputs, targets_a, targets_b = map(Variable, (imgs,targets_a, targets_b))
        # Forward the data. (Make sure data and model are on the same device.)
        logits = se_resnet50(inputs.to(device))
        
        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
#         loss = criterion(logits, labels.to(device))
        loss = mixup_criterion(criterion, logits, targets_a, targets_b, lam)
        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(se_resnet50.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()
        
        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        scheduler.step(acc)
        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    se_resnet50.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = se_resnet50(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # update logs
    if valid_acc > best_acc:
        with open(f"./se_resnet50_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        with open(f"./se_resnet50_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(se_resnet50.state_dict(), f"se_resnet50_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break
    #torch.save(se_resnet50.state_dict()

  0%|          | 0/177 [00:00<?, ?it/s]

[ Train | 001/3000 ] loss = 1.94171, acc = 0.28070


  0%|          | 0/16 [00:00<?, ?it/s]

[ Valid | 001/3000 ] loss = 1.63021, acc = 0.44883
[ Valid | 001/3000 ] loss = 1.63021, acc = 0.44883 -> best
Best model found at epoch 0, saving model


  0%|          | 0/177 [00:00<?, ?it/s]

...

[ Train | 435/3000 ] loss = 0.96913, acc = 0.48346


  0%|          | 0/16 [00:00<?, ?it/s]

[ Valid | 435/3000 ] loss = 0.74124, acc = 0.79575
[ Valid | 435/3000 ] loss = 0.74124, acc = 0.79575


  0%|          | 0/177 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Test

In [15]:
test_image_paths = split_set(os.path.join(_dataset_dir,"test"), ratio = 1, shuffle = False)
test_set = FoodDataset(test_image_paths,tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

dataset with 3347 images


In [35]:
test_set1 = FoodDataset(test_image_paths,tfm=train_tfm)
test_set2 = FoodDataset(test_image_paths,tfm=train_tfm)
test_set3 = FoodDataset(test_image_paths,tfm=train_tfm)
test_set4 = FoodDataset(test_image_paths,tfm=train_tfm)
test_set5 = FoodDataset(test_image_paths,tfm=train_tfm)
test_set_all = ConcatDataset([test_set1,test_set2,test_set3,test_set4,test_set5])
test_loader_all = DataLoader(test_set_all, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

dataset with 3347 images
dataset with 3347 images
dataset with 3347 images
dataset with 3347 images
dataset with 3347 images


# Testing and generate prediction CSV

In [36]:
# model_best = Classifier().to(device)
se_resnet50.load_state_dict(torch.load(f"se_resnet50_best.ckpt"))

se_resnet50.eval()
prediction = []

with torch.no_grad():
    for data,_ in tqdm(test_loader_all):
        test_pred = se_resnet50(data.to(device))
        try:
            test_pred_all = np.vstack([test_pred_all,test_pred.cpu().data.numpy()])
        except:
            test_pred_all = test_pred.cpu().data.numpy()
        
test_pred_all = test_pred_all.reshape(5,3347,11)
test_pred_all = np.mean(test_pred_all,axis=0) 

with torch.no_grad():
    for data,_ in tqdm(test_loader):
        test_pred = se_resnet50(data.to(device))
        try:
            test_pred_result = np.vstack([test_pred_result,test_pred.cpu().data.numpy()])
        except:
            test_pred_result = test_pred.cpu().data.numpy()
            
test_pred_final = (test_pred_result+test_pred_all)/2
test_label = np.argmax(test_pred_final, axis=1)
prediction = test_label.squeeze().tolist()

  0%|          | 0/131 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

In [18]:
se_resnet50.load_state_dict(torch.load(f"se_resnet50_best.ckpt"))
se_resnet50.eval()
prediction = []
with torch.no_grad():
    for data,_ in tqdm(test_loader):
        test_pred = se_resnet50(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()

In [37]:
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("se_resnet_mixup.csv",index = False)

# Q1. Augmentation Implementation
## Implement augmentation by finishing train_tfm in the code with image size of your choice. 
## Directly copy the following block and paste it on GradeScope after you finish the code
### Your train_tfm must be capable of producing 5+ different results when given an identical image multiple times.
### Your  train_tfm in the report can be different from train_tfm in your training code.


In [ ]:
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You need to add some transforms here.
    transforms.ToTensor(),
])

# Q2. Residual Implementation
![](https://i.imgur.com/GYsq1Ap.png)
## Directly copy the following block and paste it on GradeScope after you finish the code


In [ ]:
from torch import nn
class Residual_Network(nn.Module):
    def __init__(self):
        super(Residual_Network, self).__init__()
        
        self.cnn_layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
        )

        self.cnn_layer2 = nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
        )

        self.cnn_layer3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.BatchNorm2d(128),
        )

        self.cnn_layer4 = nn.Sequential(
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
        )
        self.cnn_layer5 = nn.Sequential(
            nn.Conv2d(128, 256, 3, 2, 1),
            nn.BatchNorm2d(256),
        )
        self.cnn_layer6 = nn.Sequential(
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(256* 32* 32, 256),
            nn.ReLU(),
            nn.Linear(256, 11)
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x1 = self.cnn_layer1(x)
        
        x1 = self.relu(x1)
        
        x2 = self.cnn_layer2(x1)
        
        x2 = self.relu(x2)
        
        x3 = self.cnn_layer3(x2)
        
        x3 = self.relu(x3)
        
        x4 = self.cnn_layer4(x3)
        
        x4 = self.relu(x4)
        
        x5 = self.cnn_layer5(x4)
        
        x5 = self.relu(x5)
        
        x6 = self.cnn_layer6(x5)
        
        x6 = self.relu(x6)
        
        # The extracted feature map must be flatten before going to fully-connected layers.
        xout = x6.flatten(1)

        # The features are transformed by fully-connected layers to obtain the final logits.
        xout = self.fc_layer(xout)
        return xout

In [71]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"dataset with {path} images",len(self.files))
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("\\")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label
